In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import numpy as np
import pandas as pd

In [3]:
# 生成示例数据
np.random.seed(42)
torch.manual_seed(42)
n_samples = 1000
n_features = 10

X = np.random.rand(n_samples, n_features)
y = 25 + 10 * np.random.rand(n_samples) + 5 * X[:, 0]  # 假设年龄与第一个特征相关，并有一定随机性

# 将数据转换为 PyTorch 张量
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)


In [4]:
X_tensor

tensor([[0.3745, 0.9507, 0.7320,  ..., 0.8662, 0.6011, 0.7081],
        [0.0206, 0.9699, 0.8324,  ..., 0.5248, 0.4319, 0.2912],
        [0.6119, 0.1395, 0.2921,  ..., 0.5142, 0.5924, 0.0465],
        ...,
        [0.0666, 0.9761, 0.9570,  ..., 0.8302, 0.9506, 0.6343],
        [0.3539, 0.6033, 0.3120,  ..., 0.2934, 0.0334, 0.8158],
        [0.4504, 0.3251, 0.4711,  ..., 0.9467, 0.3975, 0.2171]])

In [15]:
y_tensor

tensor([[30.6091],
        [28.4320],
        [29.8208],
        [34.1104],
        [30.3764],
        [38.5049],
        [27.2645],
        [35.2999],
        [36.9450],
        [33.1928],
        [34.0179],
        [33.7391],
        [38.3153],
        [30.4136],
        [34.8443],
        [29.6821],
        [26.9085],
        [30.7891],
        [27.7134],
        [28.0676],
        [29.9806],
        [28.0289],
        [37.3811],
        [29.0563],
        [37.0631],
        [27.9937],
        [37.1973],
        [37.9748],
        [35.9890],
        [31.8748],
        [25.3118],
        [37.0436],
        [36.5245],
        [35.7309],
        [34.0505],
        [30.9636],
        [35.8773],
        [28.4191],
        [31.2179],
        [30.9498],
        [34.7352],
        [28.0043],
        [31.2069],
        [31.2963],
        [33.0651],
        [35.0792],
        [27.9090],
        [25.0702],
        [35.7185],
        [37.1304],
        [36.5899],
        [32.6691],
        [37.

In [16]:
# 创建数据集
dataset = TensorDataset(X_tensor, y_tensor)

# 划分数据集为训练集和测试集
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# 定义模型
class AgePredictor(nn.Module):
    def __init__(self):
        super(AgePredictor, self).__init__()
        self.fc1 = nn.Linear(n_features, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = AgePredictor()

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [18]:
# 训练模型
n_epochs = 100
for epoch in range(n_epochs):
    print('epoch: ',epoch)
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        # if epoch == 2:
            
        #     print('outputs :', outputs)
        #     print('targets :', targets)
            
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / train_size
    print(f"Epoch {epoch+1}/{n_epochs}, Loss: {epoch_loss:.4f}")

# 评估模型
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item() * inputs.size(0)

test_loss /= test_size
print(f"Test Loss: {test_loss:.4f}")

# 打印模型预测示例
with torch.no_grad():
    for i in range(5):
        input_sample = test_dataset[i][0].unsqueeze(0)
        target_sample = test_dataset[i][1]
        prediction = model(input_sample)
        print(f"Actual Age: {target_sample.item():.2f}, Predicted Age: {prediction.item():.2f}")


epoch:  0
Epoch 1/100, Loss: 28.0015
epoch:  1
Epoch 2/100, Loss: 26.7484
epoch:  2
Epoch 3/100, Loss: 25.9486
epoch:  3
Epoch 4/100, Loss: 24.8810
epoch:  4
Epoch 5/100, Loss: 23.6577
epoch:  5
Epoch 6/100, Loss: 22.8820
epoch:  6
Epoch 7/100, Loss: 21.4838
epoch:  7
Epoch 8/100, Loss: 20.5806
epoch:  8
Epoch 9/100, Loss: 19.4293
epoch:  9
Epoch 10/100, Loss: 18.3305
epoch:  10
Epoch 11/100, Loss: 17.4997
epoch:  11
Epoch 12/100, Loss: 16.5576
epoch:  12
Epoch 13/100, Loss: 15.6019
epoch:  13
Epoch 14/100, Loss: 14.7939
epoch:  14
Epoch 15/100, Loss: 14.0016
epoch:  15
Epoch 16/100, Loss: 13.1760
epoch:  16
Epoch 17/100, Loss: 12.3910
epoch:  17
Epoch 18/100, Loss: 11.8974
epoch:  18
Epoch 19/100, Loss: 11.0872
epoch:  19
Epoch 20/100, Loss: 10.4807
epoch:  20
Epoch 21/100, Loss: 10.0015
epoch:  21
Epoch 22/100, Loss: 9.7478
epoch:  22
Epoch 23/100, Loss: 9.2484
epoch:  23
Epoch 24/100, Loss: 8.9692
epoch:  24
Epoch 25/100, Loss: 8.6800
epoch:  25
Epoch 26/100, Loss: 8.6706
epoch:  26